In [1]:
%matplotlib notebook
import sys 
sys.path.append("/Users/Hilary/Documents/Github/Py-NnK/scan")
sys.path.append("/Users/Hilary/Documents/Data/DataIceland")

import obspy
import matplotlib.pyplot as plt
import EQ
import numpy as np
from obspy import read
#from matplotlib import gridspec 
import glob, os
from obspy.geodetics.base import calc_vincenty_inverse
from scipy.optimize import curve_fit
from TravelTime import *
def e_fit(x, a, b, c):
    return a * np.exp(-b * x) +c

In [2]:
# Read a single event (with all the 18 stations)
stm = read("/Users/Hilary/Documents/Github/Data/DataIceland/20101221012407/*.sac")
# Event(source) location
ev_loc=EQ.single_ev_loc(stm)
evla=ev_loc[0]
evlo=ev_loc[1]
evdp=ev_loc[2]
# Station location 
st_loc=EQ.st_loc(stm)  
st_dis_in_m=[]
dis=[]
for i in range(len(stm)//3):   # Convert distance between stations and source from degrees to meters
    st_name = st_loc[2][i]
    distance= calc_vincenty_inverse(st_loc[0][i], st_loc[1][i], ev_loc[0], ev_loc[1])[0]
    dis.append(distance)
    st_dis_in_m.append([st_name, distance])

## Travel Time from 2D Finite-Difference(FD) Simulation

In [3]:
# Simple finite difference solver 
# Acoustic wave equation  p_tt = c^2 p_xx + src
# 2-D regular grid

nx = 700      # grid points in x 
nza = 2
nzg = 500
nz = nza+nzg      # grid points in z
nt = 1300      # number of time steps
dx = 100.0     # grid increment in x
dt = 0.005    # Time step
#c0 = 4500       # velocity (can be an array)
ca = 340
f0 = 5.0    # dominant frequency of source (Hz)
isx = nx//2  # source index x 
isz = nza + int(round(evdp*1000/dx))  # source index z
ist = 4./f0     # shifting of source time function
isnap = 10     # snapshot frequency
T = 1.0 / f0  # dominant period
nop = 5       # length of operator

# Receiver locations ######################################################
irx=np.array([],dtype=int)
for i in range(len(st_dis_in_m)):
    r = round(st_dis_in_m[i][1]/100)+ isx
    irx=np.append(irx,r)
irz= nza+np.zeros_like(irx)
seis = np.zeros((len(irx), nt))

# Initialize pressure at different time steps and the second ##############
# derivatives in each direction
p = np.zeros((nz, nx))
pold = np.zeros((nz, nx))
pnew = np.zeros((nz, nx))
pxx = np.zeros((nz, nx))
pzz = np.zeros((nz, nx))

In [4]:
# Get parameters by fitting model provided by Tryggvason et al 2002
model_dep = np.array([  0,   2,   4,   6,   9,  12,  16,  21,  32])
model_vel = np.array([3.6, 5.6, 6.4, 6.6, 6.7, 6.8, 7.0, 7.1, 7.4])
Z1, pcov = curve_fit(e_fit, model_dep[0:4], model_vel[0:4])
Z3 = np.polyfit(model_dep[3:], model_vel[3:], 1)

model_type = "Tryggvason et al 2002"  

# Initialize velocity model
c = np.zeros((nz, nx))
for i in range(0, nza):
    c[i,:] += ca
if model_type == "homogeneous":                    
    for i in range(0, nzg):
        c[nza+i,:] += c0
elif model_type == "increase linearly with depth": 
    for i in range(0, nzg):
        vmax=Z3[0]*(nzg)*dx+ 6.590
        vmin=Z1[0]+Z1[2]
        c[nza+i,:] += vmin+(vmax-vmin) * ( i/(nzg-1) )
elif model_type == "Tryggvason et al 2002":       
    for i in range(0,44):                     #   0 ~ 4.3km: (Zone1) P velocity increases exponentially
        c[nza+i,:] = 1000* (Z1[0] * np.exp(-Z1[1] * i*dx/1000) +Z1[2])
    for i in range(44, 60):                 # 4.4 ~ 5.9km: (Zone2) interpolation between Zone 1&2 
        dp=model_dep[2:4]                   # 4.4km: v=6440m/s
        vp=model_vel[2:4]
        c[nza+i,:] = 1000* np.interp(i*dx/1000, dp,vp)  
    for i in range(60, nzg):                     # 6.0 ~bottom: (Zone3) linear increase (get param 6590 by observation of data)
        c[nza+i,:] = 1000* (Z3[0]*(i-59)*dx/1000+ 6.590)
    
else:
    raise NotImplementedError
    
cmax = c.max()

In [ ]:
# Source time function Gaussian, nt + 1 as we loose the last one by diff
src = np.empty(nt + 1)
time = np.linspace(0 * dt, nt * dt, nt)
src  = -200. * (time - ist) * (f0 ** 2) * (np.exp(-1.0 * (f0 ** 2) * (time - ist) ** 2))

v = max([np.abs(src.min()), np.abs(src.max())])

# required for seismograms
ir = np.arange(len(irx))

# Output Courant criterion
print("Courant Criterion eps :")
print(cmax*dt/dx)

# Time extrapolation
for it in range(nt):
    if nop==3:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(1, nx - 1):
            pzz[:, i] = p[:, i + 1] - 2 * p[:, i] + p[:, i - 1]
        for j in range(1, nz - 1):
            pxx[j, :] = p[j - 1, :] - 2 * p[j, :] + p[j + 1, :]

    if nop==5:
        # calculate partial derivatives, be careful around the boundaries
        for i in range(2, nx - 2):
            pzz[:, i] = -1./12*p[:,i+2]+4./3*p[:,i+1]-5./2*p[:,i]+4./3*p[:,i-1]-1./12*p[:,i-2]
        for j in range(2, nz - 2):
            pxx[j, :] = -1./12*p[j+2,:]+4./3*p[j+1,:]-5./2*p[j,:]+4./3*p[j-1,:]-1./12*p[j-2,:]
                                
    pxx /= dx ** 2
    pzz /= dx ** 2

    # Time extrapolation
    pnew = 2 * p - pold + dt ** 2 * c ** 2 * (pxx + pzz)
    # Add source term at isx, isz
    pnew[isz, isx] = pnew[isz, isx] + src[it]
    pold, p = p, pnew

    # Save seismograms
    seis[ir, it] = p[irz[ir], irx[ir]]

Courant Criterion eps :
0.397571748878924


In [ ]:
# Get the time where minimum slope occurs
t_fd=[]
for i in range(0,len(seis)):
    t_fd.append(np.argmin(np.diff(seis[i]))*dt-ist)
    
# Save data to file
with open("tt_2D_FD_alphabetical.txt", "w") as tt_2D_FD:
    for tt in t_fd:
        tt_2D_FD.write("%f\n" % tt)   

## Travel Time from SAC Attributes and Correlation Picker

In [ ]:
t_sac=TravelTime_SAC(stm)
with open("tt_sac_alphabetical.txt", "w") as tt_sac_att:
    for tt in t_sac:
        tt_sac_att.write("%f\n" % tt)

In [ ]:
t_cor=TravelTime_Correlate(stm,hp=10,lp=49)
with open("tt_cor_hp10_alphabetical.txt", "w") as tt_cor:
    for tt in t_cor:
        tt_cor.write("%f\n" % tt)

## Sorting Data and Plotting

In [ ]:
import copy
st_tt = copy.deepcopy(st_dis_in_m)
for i in range(len(st_tt)):
    st_tt[i].append(t_sac[i])
    st_tt[i].append(t_cor[i])
    st_tt[i].append(t_fd[i])

In [ ]:
st_tt_sorted=sorted(st_tt, key=lambda x: x[1])  # Sort the list by x[1](x distance)
st_name=[]
dis=[]
tt_a=[]
tt_c=[]
tt_f=[]
for i in range(len(st_tt)):
    st_name.append(st_tt_sorted[i][0])
    dis.append(st_tt_sorted[i][1])
    tt_a.append(st_tt_sorted[i][2])
    tt_c.append(st_tt_sorted[i][3])
    tt_f.append(st_tt_sorted[i][4])

In [ ]:
# Filter out outliers in the correlation method
tt_c = np.asarray(tt_c)
tt_c[np.logical_or(tt_c<0, tt_c>7)] = np.nan

In [ ]:
fig, ax = plt.subplots()
plt.plot(dis, tt_a,'o-',label='Sac Attributes')
plt.plot(dis, tt_c,'o-',label='Correlation Pick')
plt.plot(dis, tt_f,'o-',label='2D FD Simulation')
plt.legend()
plt.xlabel('Distance from Epicenter (m)')
plt.ylabel('Travel Time (s)')
plt.title('Travel Time Curve')
for i,name in enumerate(st_name):
    plt.annotate(name,(dis[i],tt_a[i]),xytext=(10,-7), textcoords='offset points')

Conclusion: 2D FD simulation with 1D velocity model is enough. 

Analysis of slowness is used rather than travel time. It's a much proper function to analysis.

Increase src freqency from 2Hz to 5Hz shows a better fit between tt_sac and tt_2dFD at larger distance.
The thickness of the air layer doesn't matter.
Change nop from 3 to 5 improve the fitting of closer distances but decrease the fitting of larger distances. 